# Tutorial: Learning how to use Tune

Tuning hyperparameters is often the most expensive part of the machine learning workflow. 

Tune is built to address this, demonstrating an efficient and scalable solution for this pain point.

This tutorial will walk you through the following process:

1. Integrating Tune into your workflow
2. Setting a stopping criteria
3. Getting the best model and analyzing results

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

from ray import tune
from ray.tune.integration.keras import TuneReporterCallback
from ray.tune.examples.utils import get_iris_data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

## Visualize your data

Let's first take a look at the distribution of the dataset.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
true_data = iris['data']
true_label = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

def plot_data(X, y):
    # Visualize the data sets
    plt.figure(figsize=(16, 6))
    plt.subplot(1, 2, 1)
    for target, target_name in enumerate(names):
        X_plot = X[y == target]
        plt.plot(X_plot[:, 0], X_plot[:, 1], linestyle='none', marker='o', label=target_name)
    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[1])
    plt.axis('equal')
    plt.legend();

    plt.subplot(1, 2, 2)
    for target, target_name in enumerate(names):
        X_plot = X[y == target]
        plt.plot(X_plot[:, 2], X_plot[:, 3], linestyle='none', marker='o', label=target_name)
    plt.xlabel(feature_names[2])
    plt.ylabel(feature_names[3])
    plt.axis('equal')
    plt.legend();
    
plot_data(true_data, true_label)

Now, let's define a function that will train a model to classify this dataset.

In [ ]:
def train_on_iris():
    train_x, train_y, test_x, test_y = get_iris_data()
    model = Sequential()

    model.add(Dense(2, input_shape=(4,), activation='relu', name='fc1'))
    model.add(Dense(2, activation='relu', name='fc2'))
    model.add(Dense(3, activation='softmax', name='output'))
    optimizer = SGD(lr=0.1)
    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    # This saves the top model
    checkpoint_callback = ModelCheckpoint("model.h5", monitor='val_loss', save_best_only=True, period=3)


    # Train the model
    model.fit(
        train_x, train_y, 
        validation_data=(test_x, test_y),
        verbose=0, batch_size=5, epochs=50, callbacks=[checkpoint_callback])
    return model

In [ ]:
model = train_on_iris()
train_x, train_y, test_x, test_y = get_iris_data()
model.evaluate(train_x, train_y)

## Integrate with Tune

Now, let's use Tune to optimize a model that learns to classify Iris. This will take three steps:

1. Designate the hyperparameter space.


2. Set a callback to report results back to Tune
3. Increase the number of samples


In [ ]:
def tune_iris(config):
    train_x, train_y, test_x, test_y = get_iris_data()
    model = Sequential()

    model.add(Dense(config["dense_1"], input_shape=(4,), activation='relu', name='fc1'))
    model.add(Dense(config["dense_2"], activation='relu', name='fc2'))
    model.add(Dense(3, activation='softmax', name='output'))
    optimizer = SGD(lr=config["lr"])
    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    checkpoint_callback = ModelCheckpoint("model.h5", monitor='val_loss', save_best_only=True, period=3)


    # Train the model
    model.fit(
        train_x, train_y, 
        validation_data=(test_x, test_y),
        verbose=0, 
        batch_size=5, 
        epochs=50, 
        callbacks=[checkpoint_callback, TuneReporterCallback(freq="epoch")])
    

results = tune.run(
    tune_iris, 
    config={"lr": 0.1, "dense_1": 1, "dense_2": 0.1},
    num_samples=1,
    return_trials=False)

assert len(results.trials) == 10

## Evaluate best trained model

In [ ]:
df = results.dataframe()

logdir = results.get_best_logdir("keras_info:val_loss", mode="min")

# import keras.models
from keras.models import load_model
model = load_model(logdir + "/model.h5")

train_data, train_labels, _, _ = get_iris_data()
plot_data(train_data, train_labels.argmax(1))

In [ ]:
res = model.evaluate(train_data, train_labels)
print("Loss is {}".format(res[0]))
print("Accuracy is {}".format(res[1]))
predicted_label = model.predict(train_data)
plot_data(train_data, predicted_label.argmax(1))

## Use Tensorboard for results

In [ ]:
! ls {logdir}

In [ ]:
! tensorboard --logdir {logdir}